<a href="https://colab.research.google.com/github/cheffjiu/pytorch-tutorials-zh/blob/main/buildmodel_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

[Learn the Basics](https://github.com/cheffjiu/pytorch-tutorials-zh/blob/main/intro.ipynb) \|\|
[Quickstart](https://github.com/cheffjiu/pytorch-tutorials-zh/blob/main/quickstart_tutorial.ipynb) \|\|
[Tensors](https://github.com/cheffjiu/pytorch-tutorials-zh/blob/main/tensorqs_tutorial.ipynb) \|\| [Datasets &
DataLoaders](https://github.com/cheffjiu/pytorch-tutorials-zh/blob/main/data_tutorial.ipynb) \|\|
[Transforms](https://github.com/cheffjiu/pytorch-tutorials-zh/blob/main/transforms_tutorial.ipynb) \|\| **Build Model** \|\|
[Autograd](https://github.com/cheffjiu/pytorch-tutorials-zh/blob/main/autogradqs_tutorial.ipynb) \|\|
[Optimization](https://github.com/cheffjiu/pytorch-tutorials-zh/blob/main/optimization_tutorial.ipynb) \|\| [Save & Load
Model](https://github.com/cheffjiu/pytorch-tutorials-zh/blob/main/saveloadrun_tutorial.ipynb)

Build the Neural Network(构建神经网络)
========================

神经网络由对数据执行操作的层/模块组成。 [torch.nn](https://pytorch.org/docs/stable/nn.html)命名空间提供了构建自己的神经网络所需的所有基础组件。 PyTorch中的每个模块都是[nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html)的子类。
神经网络本身就是一个模块，它由其他模块（层）组成。这种嵌套结构便于轻松构建和管理复杂的架构。

在接下来的章节中，我们将构建一个神经网络，对FashionMNIST数据集中的图像进行分类。


In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

Get Device for Training（获取训练设备）
=======================

我们希望能够在诸如CUDA、MPS、MTIA或XPU等[加速器](https://pytorch.org/docs/stable/torch.html#accelerators)上训练模型。如果当前的加速器可用，我们将使用它。否则，我们就使用CPU。


In [ ]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Define the Class（定义模型类）
================

我们通过继承`nn.Module`来定义神经网络，并在`__init__`中初始化神经网络层。每个`nn.Module`子类都在`forward`方法中实现对输入数据的操作。

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

我们创建一个`NeuralNetwork`实例，将其移动到`device`上，并打印其结构。


In [ ]:
model = NeuralNetwork().to(device)
print(model)

要使用该模型，我们将输入数据传递给它。这会执行模型的 `forward` 方法， 同时也有[
operations](https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866).
不要直接调用`model.forward()`！

对输入调用模型会返回一个二维张量，其中维度dim=0对应每个类别的10个原始预测值的每个输出，维度dim=1对应每个输出的各个值。我们通过将其传入`nn.Softmax`模块的实例来获得预测概率。


In [ ]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

------------------------------------------------------------------------


Model Layers(模型层)
============

让我们剖析FashionMNIST模型中的各层。为了说明这一点，我们将取一个包含3张28x28尺寸图像的小批量样本，看看当它通过网络时会发生什么。


In [ ]:
input_image = torch.rand(3,28,28)
print(input_image.size())

nn.Flatten
==========

我们初始化
[nn.Flatten](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html)
层，将每个2D的28x28图像转换为一个包含784个像素值的连续数组（小批量维度（在维度0处）保持不变）。


In [ ]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

nn.Linear
=========

[线性层](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)是一个模块，它使用存储的weights和bias对输入进行线性变换。


In [ ]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

nn.ReLU
=======

非线性激活函数构建了模型输入与输出之间的复杂映射关系。它们在线性变换之后应用，以引入“非线性”，帮助神经网络学习各种各样的现象。

在这个模型中，我们在线性层之间使用[nn.ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html)，但也有其他激活函数可以在你的模型中引入非线性。


In [ ]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

nn.Sequential
=============

[nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html) 是一个有序的模块容器。数据按照定义的相同顺序依次通过所有模块。你可以使用顺序容器来快速搭建一个类似 `seq_modules` 的网络。


In [ ]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

nn.Softmax
==========

神经网络的最后一个线性层返回 [logits]
- raw values 在 \[-∞, +∞\] 中- 这些值会被传递给[nn.Softmax](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html)模块。`logits`会被缩放到[0, 1]范围内的值，这些值代表模型对每个类别的预测概率。`dim`参数表示值必须沿其求和为1的维度。


In [ ]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

Model Parameters(模型参数)
================

神经网络内部的许多层都是**参数化的**，即具有在训练过程中进行优化的相关权重和偏差。继承 `nn.Module` 会自动跟踪模型对象内部定义的所有字段，并通过模型的 `parameters()` 或 `named_parameters()` 方法访问所有参数。

在这个例子中，我们遍历每个参数，并打印其大小以及其值的预览。


In [ ]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

------------------------------------------------------------------------


Further Reading
===============

-   [torch.nn API](https://pytorch.org/docs/stable/nn.html)
